In [1]:
#install hugging face transformers
!pip install transformers

You should consider upgrading via the '/Users/admin/.pyenv/versions/3.7.7/envs/finbert/bin/python -m pip install --upgrade pip' command.


In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

#import pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

#import pretrained model
#model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [5]:
#specify path for saving pretrained models
path = '/Users/admin/code/Michiel-DK/finbert/notebooks/'

In [20]:
#save weights model
torch.save(tokenizer, f"{path}pretrained_tokenizer.pth")

#save weights model
torch.save(model.state_dict(), f"{path}pytorch_model.bin")


In [22]:
#load weights tokenizer
model = AutoModelForSequenceClassification.from_pretrained(f"{path}", local_files_only=True, num_labels=3)
#model.load_state_dict(torch.load(path, map_location=torch.device('cpu')), strict=False)

#load weights model
#model.load_state_dict(torch.load(path, map_location=torch.device('cpu')), strict=False)

In [51]:
# original saved file with DataParallel
checkpoint = torch.load(path, map_location=torch.device('cpu'))

# create new OrderedDict that does not contain `module.`
from collections import OrderedDict

new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = k.replace("module.", "") # remove `module.`
    new_state_dict[name] = v

# load params
model.load_state_dict(new_state_dict, strict=False)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [49]:
model.load_state_dict(torch.load('/Users/admin/code/Michiel-DK/finbert/notebooks/pretrained.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [45]:
!pwd

/Users/admin/code/Michiel-DK/finbert/notebooks


In [44]:
type(tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [1]:
import pandas as pd

headlines_df = pd.read_csv('../data/analyst_ratings_processed.csv')
headlines_df = headlines_df.sample(10, random_state=42)

In [15]:
headlines_df.columns[0]

'Unnamed: 0'

In [2]:
import numpy as np

headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,1])

print(headlines_list[:5])

['Cedar Shopping Centers Closes Purchase of New Jersey Center for $26.3M ', 'eFuture Announces Appointment of Ms. Ping Yu As CFO', 'Courier Corp Holder Gamco Reports 6.97%, Up From 0.05% Pn 12/31/2014', 'ONEOK Partners to Participate in Bakken Day', 'Marsh Subsidiary Marsh & McLennan To Acquire Benefits Advisory Group']


In [27]:
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')

In [28]:
outputs = model(**inputs)
print(outputs.logits)

tensor([[ 2.0557, -1.4729, -1.7830],
        [-0.2203, -1.6770,  2.2643],
        [ 0.2181,  1.3837, -1.6399],
        [ 2.1141, -2.0579, -1.3233],
        [-0.2195, -1.7844,  2.4427],
        [ 0.4874, -2.4585,  2.1729],
        [-0.7994, -1.0177,  2.0862],
        [-0.3778, -1.8577,  2.5023],
        [-0.6469, -1.5176,  2.5074],
        [ 0.4043, -2.4181,  2.0182]], grad_fn=<AddmmBackward>)


In [29]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.9516, 0.0279, 0.0205],
        [0.0756, 0.0176, 0.9068],
        [0.2292, 0.7351, 0.0357],
        [0.9546, 0.0147, 0.0307],
        [0.0644, 0.0135, 0.9222],
        [0.1551, 0.0082, 0.8368],
        [0.0507, 0.0408, 0.9085],
        [0.0525, 0.0120, 0.9355],
        [0.0402, 0.0168, 0.9429],
        [0.1644, 0.0098, 0.8258]], grad_fn=<SoftmaxBackward>)


In [2]:
positive

NameError: name 'positive' is not defined

In [30]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

In [31]:
df.sort_values(by='Positive', ascending=False)

,Headline,Positive,Negative,Neutral
3,Asterias Provides 6 Mo. Data Readout From Its ...,0.954589,0.014721,0.030690
0,"Courier Corp Holder Gamco Reports 6.97%, Up Fr...",0.951596,0.027925,0.020479
2,UPDATE: Transocean and Chevron Moving off Lows...,0.229157,0.735099,0.035745
9,Top Stocks In The Surety & Title Insurance Ind...,0.164415,0.009777,0.825807
5,ONEOK Partners to Participate in Bakken Day,0.155090,0.008151,0.836759
1,Cedar Shopping Centers Closes Purchase of New ...,0.075591,0.017613,0.906796
4,Marsh Subsidiary Marsh & McLennan To Acquire B...,0.064367,0.013459,0.922174
7,The Deal Says Citrix Is Not Exploring A Possib...,0.052512,0.011955,0.935533
6,eFuture Announces Appointment of Ms. Ping Yu A...,0.050713,0.040770,0.908517
8,Shares of Broadsoft to Resume Trade at 4:35PM EST,0.040232,0.016843,0.942925


In [3]:
# Read JSON
import json
from datetime import datetime
#json_file_path = f"../data/{datetime.today().strftime('%Y-%m-%d')}.json"
json_file_path = "../data/2021-11-21.json"


with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [11]:
list(dictionary.keys())[0]

'HALO'

In [56]:
new_ls = []
#for ticker in contents.keys():
for ticker in ['HALO', 'OLED']:
    for item in contents[ticker]['news']:
        item.append(ticker)
        new_ls.append(item)
        

In [57]:
round(len(new_ls)/300)

1

In [20]:
import numpy as np
headlines_array = np.array(new_ls)
np.random.shuffle(headlines_array)


In [21]:
headlines_array[0]

array(['Jul-19-21',
       'Hedge Funds Are Cashing Out Of Halozyme Therapeutics, Inc. (HALO)',
       'HALO'], dtype='<U257')

In [58]:
from tqdm import tqdm
import torch
import pandas as pd

df = pd.DataFrame(columns = ["date", "ticker","headline", "positive", "negative", "neutral"])

for group in tqdm(np.array_split(headlines_array, round(len(headlines_array)/300))):
    headlines_list = list(group[:,1])
    ticker_list = list(group[:,-1])
    date_list = list(group[:,0])

    inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()

    table = {'date':date_list,
        'ticker':ticker_list,
        'headline':headlines_list,
            "positive":positive,
            "negative":negative, 
            "neutral":neutral}
    
    df = df.append(pd.DataFrame.from_dict(table))
df = df.reset_index(drop=True)

100%|██████████| 1/1 [02:42<00:00, 162.89s/it]


In [32]:
df['ticker']

KeyError: 'ticker'

In [21]:
df.groupby('ticker').mean().sort_values(by='positive')

,positive,negative,neutral
ticker,,,
OLED,0.236796,0.128001,0.635203
HALO,0.352239,0.108665,0.539095


In [5]:
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs)


In [ ]:
print(outputs.logits.shape)

In [ ]:
import torch
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [ ]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Ticker':ticker_list,
    'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Ticker","Headline", "Positive", "Negative", "Neutral"])

In [ ]:
df